## Cluster configurations

This notebook will take the trajectory and go through every frame, every ion-small molecule \
bound configuration and treat them as a datapoint. Cluster the dtapoints based on distance \
and find the representative medoid structure of the clusters. These medoids are teh input \
for the FEP (Free Energy Perturbation) later. 

In [1]:
# Imports

import numpy as np
import pandas as pd
import MDAnalysis as mda
from MDAnalysis.lib.distances import distance_array, minimize_vectors
import MDAnalysis.analysis.rdf as rdf

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances

import matplotlib.pyplot as plt

from kneed import KneeLocator

import warnings
from joblib import Parallel, delayed
import itertools # To flatten the list of lists from parallel workers

warnings.filterwarnings('ignore')


### Load files

In [3]:
data_dir = "../simulation_data/"

PSF_FILE = data_dir+"step3_input.psf"
DCD_FILE = data_dir+"step5_100.dcd"


# Just to get number of frames
u_check = mda.Universe(PSF_FILE, DCD_FILE)
NUM_FRAMES = len(u_check.trajectory)
SKIP_FRAMES = 1 
del u_check # Free up memory


N_per_group = 4 # Take only the 4 closest of a neighbor group
TOTAL_FEATURES = N_per_group * 2 # There are two types of neighbors- water and GLP2

N_CORES = 128 # number of cores

# My atom sleections
ref_sel = "resname BE" # Central ion
glp_sel = "resname GLP2 and type PL" # Atom from solute
water_sel = "resname TIP3 and type OT" # Atom from neighboring water 
cutoff_radius = 5.0 #(A) Cutoff distance for neighbor search


In [5]:
def get_dist_features_for_frame(u, ref_atoms, glp_atoms, water_atoms, N, ts):
    frameNum = ts.frame
    all_frame_data = []
    
    for i in range(len(ref_atoms)):
        ref_atom = ref_atoms[i:i+1]
        ref_resid = ref_atom.resids[0]
        
        glp_dists, glp_info = [], []
        if len(glp_atoms) > 0:
            dist_arr = distance_array(ref_atom.positions, glp_atoms.positions, box=u.dimensions)[0]
            sorted_indices = np.argsort(dist_arr)[:N]
            glp_dists.extend(dist_arr[sorted_indices])

        water_dists, water_info = [], []
        if len(water_atoms) > 0:
            dist_arr = distance_array(ref_atom.positions, water_atoms.positions, box=u.dimensions)[0]
            sorted_indices = np.argsort(dist_arr)[:N]
            water_dists.extend(dist_arr[sorted_indices])

        # Pad with a large number if not enough neighbors are found within cutoff_radius
        glp_dists.extend([999.9] * (N - len(glp_dists)))
        water_dists.extend([999.9] * (N - len(water_dists)))
            
        final_features = np.array(glp_dists + water_dists)

        all_frame_data.append({
            'frame': frameNum,
            'be_resid': ref_resid,
            'features': final_features,
        })
        
    return all_frame_data


In [6]:

# This is the NEW worker function that each of the 128 cores will run.
# It takes a frame index, creates its own Universe, and calls the function above.
def process_frame(frame_index):
    # Each worker creates its own Universe object
    u = mda.Universe(PSF_FILE, DCD_FILE)
    ts = u.trajectory[frame_index]
    
    # Static selections made once per worker
    ref_atoms_static = u.select_atoms(ref_sel)
    glp_atoms_static = u.select_atoms(glp_sel)
    
    # Dynamic selection of water within the cutoff for this frame
    water_atoms_in_frame = u.select_atoms(f"({water_sel}) and around {cutoff_radius} group ref_atoms_static", ref_atoms_static=ref_atoms_static)
    
    # Generate and return features
    return get_dist_features_for_frame(u, ref_atoms_static, glp_atoms_static, water_atoms_in_frame, N=N_per_group, ts=ts)


In [7]:

# Run with joblib for parallel implementation
print(f"Starting parallel analysis on {N_CORES} cores for {NUM_FRAMES} frames...")

# The Parallel command distributes the execution of process_frame over N_CORES
results_list_of_lists = Parallel(n_jobs=N_CORES, verbose=10, backend="loky")(
    delayed(process_frame)(i) for i in range(0,NUM_FRAMES, SKIP_FRAMES)
)

print("\nParallel processing complete. Assembling DataFrame...")



Step 2: Starting parallel analysis on 128 cores for 120000 frames...


[Parallel(n_jobs=128)]: Using backend LokyBackend with 128 concurrent workers.
/scratch/zt1/project/energybio/user/jbodosa/data/exec/miniconda3/envs/py3_mda/lib/python3.12/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
/scratch/zt1/project/energybio/user/jbodosa/data/exec/miniconda3/envs/py3_mda/lib/python3.12/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanal


Parallel processing complete. Assembling DataFrame...


[Parallel(n_jobs=128)]: Done   1 tasks      | elapsed:  1.3min
[Parallel(n_jobs=128)]: Done   8 out of  60 | elapsed:  1.3min remaining:  8.4min
[Parallel(n_jobs=128)]: Done  15 out of  60 | elapsed:  1.3min remaining:  3.9min
[Parallel(n_jobs=128)]: Done  22 out of  60 | elapsed:  1.3min remaining:  2.2min
[Parallel(n_jobs=128)]: Done  29 out of  60 | elapsed:  1.3min remaining:  1.4min
[Parallel(n_jobs=128)]: Done  36 out of  60 | elapsed:  1.3min remaining:   51.7s
[Parallel(n_jobs=128)]: Done  43 out of  60 | elapsed:  1.3min remaining:   30.7s
[Parallel(n_jobs=128)]: Done  50 out of  60 | elapsed:  1.3min remaining:   15.5s
[Parallel(n_jobs=128)]: Done  57 out of  60 | elapsed:  1.3min remaining:    4.1s
[Parallel(n_jobs=128)]: Done  60 out of  60 | elapsed:  1.3min finished


In [8]:

# flatten lists into a single list
all_features_list = list(itertools.chain.from_iterable(results_list_of_lists))

features_df = pd.DataFrame(all_features_list)
print(f"Generated {len(features_df)} feature vectors, each with {TOTAL_FEATURES} features.")

features_dat = np.vstack(features_df['features'].values)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features_dat)


Generated 1200 feature vectors, each with 8 features.


In [ ]:
# Find the best k from k-means clustering

inertia = []
k_range = range(2, 11) 

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(scaled_features)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, inertia, marker='o', linestyle='--')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.xticks(k_range)
plt.grid(True)
plt.savefig("elbow_plot.png")



Step 3: Finding optimal k using the Elbow (Knee) Method...


In [ ]:

kl = KneeLocator(k_range, inertia, curve="convex", direction="decreasing")
optimal_k = kl.elbow 

if optimal_k is None:
    print("Could not automatically find an elbow. Using fallback k=8.")
    optimal_k = 8
else:
    print(f"\nOptimal k automatically found: {optimal_k}")


# Run clustering with optimal k, get medoids
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init='auto')
features_df['cluster'] = kmeans.fit_predict(scaled_features)

medoid_rows = []
for cluster_id in sorted(features_df['cluster'].unique()):
    cluster_indices = features_df.index[features_df['cluster'] == cluster_id]
    points_in_cluster = scaled_features[cluster_indices]
    distance_matrix = pairwise_distances(points_in_cluster)
    sum_of_distances = distance_matrix.sum(axis=1)
    medoid_local_index = np.argmin(sum_of_distances)
    medoid_original_index = cluster_indices[medoid_local_index]
    medoid_rows.append(features_df.loc[medoid_original_index])

medoids_df = pd.DataFrame(medoid_rows)
print("\n--- Medoids for each cluster ---")
print(medoids_df[['frame', 'be_resid', 'cluster']])



In [ ]:
# Calculating RDFs for medoids to determine cutoff
first_minima_distances = []

u = mda.Universe(PSF_FILE, DCD_FILE)

for index, medoid_info in medoids_df.iterrows():
    medoid_frame = int(medoid_info['frame'])
    medoid_be_resid = int(medoid_info['be_resid'])
    
    u.trajectory[medoid_frame]
    
    g1 = u.select_atoms(f"resid {medoid_be_resid} and resname BE")
    g2 = u.select_atoms("resname GLP2 and (type PL or type O* or type C*)")
    
    rdf_analysis = rdf.InterRDF(g1, g2, nbins=150, range=(0.0, 15.0))
    rdf_analysis.run()
    
    first_peak_index = np.argmax(rdf_analysis.results.rdf)
    first_minimum_index = -1
    for i in range(first_peak_index, len(rdf_analysis.results.rdf) - 1):
        if rdf_analysis.results.rdf[i] < rdf_analysis.results.rdf[i+1]:
            first_minimum_index = i
            break
            
    if first_minimum_index != -1:
        dist = rdf_analysis.results.bins[first_minimum_index]
        first_minima_distances.append(dist)
        print(f"  - Medoid (Frame {medoid_frame}, BE {medoid_be_resid}): First GLP2 RDF minimum found at {dist:.2f} Å")

if not first_minima_distances:
    rdf_cutoff = 7.0 # Deafault distance cutoff 7A 
    print(f"\nCould not determine RDF minima. Using default cutoff: {rdf_cutoff:.2f} Å")
else:
    rdf_cutoff = np.mean(first_minima_distances) + 3 # 3A buffer needed for complete neighbors
    print(f"\nDetermined average RDF cutoff for visualization: {rdf_cutoff:.2f} Å")


In [ ]:


# Extract PDB file for each medoid
for index, medoid_info in medoids_df.iterrows():
    medoid_frame = int(medoid_info['frame'])
    medoid_be_resid = int(medoid_info['be_resid'])
    cluster_id = int(medoid_info['cluster'])
    
    print(f"\nProcessing medoid for Cluster {cluster_id} (Frame: {medoid_frame}, BE Resid: {medoid_be_resid})...")
    
    ts = u.trajectory[medoid_frame]
    be_atom = u.select_atoms(f"resid {medoid_be_resid} and resname BE")
    potential_neighbors = u.select_atoms(f"((resname GLP2) or (resname TIP3)) and around 15 group be_atom", be_atom=be_atom)
    group_to_process = be_atom + potential_neighbors.residues.atoms
    group_copy = group_to_process.copy()
    
    center_point = be_atom.center_of_geometry()
    group_copy.atoms.translate(-center_point)
    
    box_vectors = u.dimensions
    neighbor_residues = group_copy.select_atoms("not resname BE").residues
    
    for res in neighbor_residues:
        original_pos = res.atoms.center_of_geometry()
        mic_vector = original_pos.copy()
        minimize_vectors(mic_vector, box_vectors)
        translation_vector = mic_vector - original_pos
        res.atoms.translate(translation_vector)

    final_atoms_list = [group_copy.select_atoms("resname BE")]
    neighbors_final = group_copy.select_atoms("not resname BE").residues
    for res in neighbors_final:
        com_distance = np.linalg.norm(res.atoms.center_of_geometry())
        if com_distance <= rdf_cutoff: # Using the new RDF cutoff
            final_atoms_list.append(res.atoms)
            
    final_atom_group = mda.Merge(*[ag.atoms for ag in final_atoms_list])
    
    output_pdb_filename = f"medoid_cluster_{cluster_id}_frame_{medoid_frame}_filtered_{rdf_cutoff:.1f}A.pdb"
    final_atom_group.atoms.write(output_pdb_filename)
    
    print(f"✅ Wrote {final_atom_group.atoms.n_atoms} filtered atoms to '{output_pdb_filename}'")

